# Creating FABnet IPv4 Network: Fully Automatic Configuration

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet using FABlib's the fully automatic configuration.   


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,47556e8d-33f2-4482-b0b3-30f17ecb271b
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,rodrigo_0000140592
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and each is automatically connected to a FABnetv4 network.  Fully automatic configuration does not require the user to manually add or configure interfaces. Instead, the `node.add_fabnet()` function is called directly on the node.  The call adds an interface and connects it to a FABnet network on the local site.  If multiple nodes reside on the same site, they will be connected to the same FABnet network.   The interface's mode is set to `auto` enabling an automatic configuration of the IP addresses during the post boot phase.

This is the simplest way to connect your nodes to FABnet and is recommended for anyone who does not need custom control over IPs and routes.


In [2]:
slice_name = 'Cassandra'
[site1,site2,site3,site4] = fablib.get_random_sites(count=4)
print(f"Sites: {site1}, {site2}, {site3}, {site4}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
node4_name = 'Node4'

network1_name='net1'
network2_name='net2'
network3_name='net3'
network4_name='net4'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'
node4_nic_name = 'nic4'

Sites: CLEM, INDI, GPN, STAR


## Configuring Interfaces of All Nodes of Slices

In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Networks
net1 = slice.add_l3network(name=network1_name, type='IPv4')
net2 = slice.add_l3network(name=network2_name, type='IPv4')
net3 = slice.add_l3network(name=network3_name, type='IPv4')
net4 = slice.add_l3network(name=network4_name, type='IPv4')


# Node1
node1 = slice.add_node(name=node1_name, site=site1, cores=16, ram=32, disk=30, image='default_ubuntu_20')
iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)
node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())

# Node2
node2 = slice.add_node(name=node2_name, site=site2, cores=16, ram=32, disk=30, image='default_ubuntu_20')
iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
iface2.set_mode('auto')
net2.add_interface(iface2)
node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net2.get_gateway())

# Node3
node3 = slice.add_node(name=node3_name, site=site3, cores=16, ram=32, disk=30, image='default_ubuntu_20')
iface3  = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]
iface3.set_mode('auto')
net3.add_interface(iface3)
node3.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net3.get_gateway())

# Node4
node4 = slice.add_node(name=node4_name, site=site4, cores=16, ram=32, disk=30, image='default_ubuntu_20')
iface4  = node4.add_component(model='NIC_Basic', name=node4_nic_name).get_interfaces()[0]
iface4.set_mode('auto')
net4.add_interface(iface4)
node4.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net4.get_gateway())

#Submit Slice Request
slice_id = slice.submit();


Retry: 12, Time: 342 sec


ID,570ebb82-d528-47d4-a641-46fa46359d93
Name,Cassandra
Lease Expiration (UTC),2024-03-22 22:26:39 +0000
Lease Start (UTC),2024-03-21 22:26:40 +0000
Project ID,47556e8d-33f2-4482-b0b3-30f17ecb271b
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
4b67dc83-287c-4bb1-9f05-d9bc79b3abe6,Node1,16,32,100,default_ubuntu_20,qcow2,clem-w3.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe6e:eeec,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe6e:eeec,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7c003013-74b4-46b8-8468-4d2829673c94,Node2,16,32,100,default_ubuntu_20,qcow2,indi-w3.fabric-testbed.net,INDI,ubuntu,2001:18e8:fff0:3:f816:3eff:fe10:2dd5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:fe10:2dd5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e464106d-9cb9-4886-8b16-21adb434bcab,Node3,16,32,100,default_ubuntu_20,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe5c:191,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe5c:191,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5c139b03-9da8-4395-90ca-3b8a22f904e7,Node4,16,32,100,default_ubuntu_20,qcow2,star-w4.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe35:47c8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe35:47c8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
80fd8e32-5200-4ecc-9912-3e0fb244a352,net1,L3,FABNetv4,CLEM,10.136.8.0/24,10.136.8.1,Active,
ca04bab6-098a-4b9d-b6ce-e1bb6cc543a1,net2,L3,FABNetv4,INDI,10.140.1.0/24,10.140.1.1,Active,
d1ea3593-d4cf-4310-867f-4bf90b1392ea,net3,L3,FABNetv4,GPN,10.135.3.0/24,10.135.3.1,Active,
e28a7c6a-f5eb-43f4-8d25-bd66701e08b4,net4,L3,FABNetv4,STAR,10.129.133.0/24,10.129.133.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic1-p1,p1,Node1,net1,100,auto,,06:49:81:02:94:C9,enp6s0,enp6s0,10.136.8.2,4
Node2-nic2-p1,p1,Node2,net2,100,auto,,0E:EF:BF:83:B9:B6,enp7s0,enp7s0,10.140.1.2,4
Node3-nic3-p1,p1,Node3,net3,100,auto,,12:29:23:4F:18:2B,enp6s0,enp6s0,10.135.3.2,6
Node4-nic4-p1,p1,Node4,net4,100,auto,,02:67:7F:D4:71:C5,enp7s0,enp7s0,10.129.133.2,4



Time to print interfaces 342 seconds


## Run the Experiment - Ping test

With automatic configuration the slice is ready for experimentation after it becomes active.  Note that automatic configuration works well when saving slices to a file and reinstantiating the slice.  Configuration tasks can be stored in the saved slice, reducing the complexity of notebooks and other runtime steps.

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [4]:
node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name) 
node3 = slice.get_node(name=node3_name)        
node4 = slice.get_node(name=node4_name) 

node1_addr = node1.get_interface(network_name=network1_name).get_ip_addr()
node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()
node3_addr = node3.get_interface(network_name=network3_name).get_ip_addr()
node4_addr = node4.get_interface(network_name=network4_name).get_ip_addr()

node1_net = node1.get_interface(network_name=network1_name).network.get_subnet()
node2_net = node2.get_interface(network_name=network2_name).network.get_subnet()
node3_net = node3.get_interface(network_name=network3_name).network.get_subnet()
node4_net = node4.get_interface(network_name=network4_name).network.get_subnet()

stdout, stderr = node1.execute(f'ping -c 1 {node2_addr}')
stdout, stderr = node1.execute(f'ping -c 1 {node3_addr}')
stdout, stderr = node1.execute(f'ping -c 1 {node4_addr}')

node1_if_name = node1.get_interface(network_name=network1_name).get_device_name()
node2_if_name = node2.get_interface(network_name=network2_name).get_device_name()
node3_if_name = node3.get_interface(network_name=network3_name).get_device_name()
node4_if_name = node4.get_interface(network_name=network4_name).get_device_name()

print("IP Addr Node 1: "+str(node1_addr))
print("IP Addr Node 2: "+str(node2_addr))
print("IP Addr Node 3: "+str(node3_addr))
print("IP Addr Node 4: "+str(node4_addr))

print("IF Name Node 1: "+str(node1_if_name))
print("IF Name Node 2: "+str(node2_if_name))
print("IF Name Node 3: "+str(node3_if_name))
print("IF Name Node 4: "+str(node4_if_name))


PING 10.140.1.2 (10.140.1.2) 56(84) bytes of data.
64 bytes from 10.140.1.2: icmp_seq=1 ttl=60 time=36.7 ms

--- 10.140.1.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 36.737/36.737/36.737/0.000 ms
PING 10.135.3.2 (10.135.3.2) 56(84) bytes of data.
64 bytes from 10.135.3.2: icmp_seq=1 ttl=58 time=85.7 ms

--- 10.135.3.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 85.682/85.682/85.682/0.000 ms
PING 10.129.133.2 (10.129.133.2) 56(84) bytes of data.
64 bytes from 10.129.133.2: icmp_seq=1 ttl=61 time=31.3 ms

--- 10.129.133.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 31.295/31.295/31.295/0.000 ms
IP Addr Node 1: 10.136.8.2
IP Addr Node 2: 10.140.1.2
IP Addr Node 3: 10.135.3.2
IP Addr Node 4: 10.129.133.2
IF Name Node 1: enp6s0
IF Name Node 2: enp7s0
IF Name Node 3: enp6s0
IF Name Node 4: enp7s0


## Configuring Default Routes to avoid 10.96.0.1:443 network unreachable

In [5]:
try:
    
    #Configuring Default route in nodes
    stdout, stderr = node1.execute(f'sudo ip route add default via {node1_addr} dev {node1_if_name} proto dhcp src {node1_addr} metric 100')
    print (stdout)
    
    stdout, stderr = node2.execute(f'sudo ip route add default via {node2_addr} dev {node2_if_name} proto dhcp src {node2_addr} metric 100')
    print (stdout)
    
    stdout, stderr = node3.execute(f'sudo ip route add default via {node3_addr} dev {node3_if_name} proto dhcp src {node3_addr} metric 100')
    print (stdout)
    
    stdout, stderr = node4.execute(f'sudo ip route add default via {node4_addr} dev {node4_if_name} proto dhcp src {node4_addr} metric 100')
    print (stdout)
    
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Inserting values into Hosts file of each machine

In [6]:
# Dicionário com os objetos de nós e seus endereços IP correspondentes
nodes = {
    node1: node1_addr,
    node2: node2_addr,
    node3: node3_addr,
    node4: node4_addr
}

# Comando para adicionar as informações no /etc/hosts de cada nó
for node, address in nodes.items():
    # Criar uma lista de entradas para todos os outros nós
    other_nodes = [f"{addr} {name.get_name()}" for name, addr in nodes.items() if name != node]

    # Adicionar a entrada do próprio nó no início da lista de outras entradas
    own_node_entry = f"{address} {node.get_name()}"
    all_entries = [own_node_entry] + other_nodes

    # Combinar todas as entradas em uma única string, separando por quebras de linha
    entries = '\n'.join(all_entries)

    # Comando para adicionar as informações no /etc/hosts do nó atual
    command = f"sudo sh -c 'echo \"{entries}\" > /etc/hosts'"
    
    # Executar o comando no nó atual
    node.execute(command)
    command = 'echo "nameserver 2001:4860:4860::8888" | sudo tee -a /etc/resolv.conf >/dev/null'
    node.execute(command)

## Installing NetData Monitoring for Tenants

In [ ]:
try:
    stdout, stderr = node1.execute(f"curl https://my-netdata.io/kickstart.sh > /tmp/netdata-kickstart.sh && sh /tmp/netdata-kickstart.sh")
    stdout, stderr = node2.execute(f"curl https://my-netdata.io/kickstart.sh > /tmp/netdata-kickstart.sh && sh /tmp/netdata-kickstart.sh")
    stdout, stderr = node3.execute(f"curl https://my-netdata.io/kickstart.sh > /tmp/netdata-kickstart.sh && sh /tmp/netdata-kickstart.sh")
    stdout, stderr = node4.execute(f"curl https://my-netdata.io/kickstart.sh > /tmp/netdata-kickstart.sh && sh /tmp/netdata-kickstart.sh")
    
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/netdata.conf", remote_file_path="netdata.conf")
    file_attributes = node2.upload_file(local_file_path="/home/fabric/work/kubernetes_files/netdata.conf", remote_file_path="netdata.conf")
    file_attributes = node3.upload_file(local_file_path="/home/fabric/work/kubernetes_files/netdata.conf", remote_file_path="netdata.conf")
    file_attributes = node4.upload_file(local_file_path="/home/fabric/work/kubernetes_files/netdata.conf", remote_file_path="netdata.conf")
    
    stdout, stderr = node1.execute(f"sudo mv netdata.conf /etc/netdata/netdata.conf")
    stdout, stderr = node2.execute(f"sudo mv netdata.conf /etc/netdata/netdata.conf")
    stdout, stderr = node3.execute(f"sudo mv netdata.conf /etc/netdata/netdata.conf")
    stdout, stderr = node4.execute(f"sudo mv netdata.conf /etc/netdata/netdata.conf")
    
    stdout, stderr = node1.execute(f"sudo systemctl restart netdata")
    stdout, stderr = node2.execute(f"sudo systemctl restart netdata")
    stdout, stderr = node3.execute(f"sudo systemctl restart netdata")
    stdout, stderr = node4.execute(f"sudo systemctl restart netdata")
    

except Exception as e:
    print(f"Exception: {e}")
 

## Configuring Kubernetes

In [7]:
try:
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_control_plane-v2.sh", remote_file_path="config_control_plane-v2.sh")

    stdout, stderr = node1.execute(f"chmod +x config_control_plane-v2.sh && ./config_control_plane-v2.sh")

except Exception as e:
    print(f"Exception: {e}")
 



Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2810 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [426 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2683 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:12 http://nova.clouds.archive.ubuntu.com/ubuntu foc

### Starting Control Plane - Master _Node1_

In [8]:
import re
key_string = ''
try:
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_control_plane-v2.sh", remote_file_path="start_control_plane-v2.sh")
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/tigera-operator.yaml", remote_file_path="tigera-operator.yaml")
    
    key_string, stderr = node1.execute(f"chmod +x start_control_plane-v2.sh && ./start_control_plane-v2.sh {node1_net} {node1_addr}")
    _, _ = node1.execute(f"kubectl apply -f tigera-operator.yaml")
except Exception as e:
    print(f"Exception: {e}")

W0321 22:34:10.338103    7215 preflight.go:56] [reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
W0321 22:34:10.338164    7215 removeetcdmember.go:106] [reset] No kubeadm config, using etcd pod spec to get data directory
[reset] Deleted contents of the etcd data directory: /var/lib/etcd
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
[reset] Deleting contents of directories: [/etc/kubernetes/manifests /var/lib/kubelet /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]

The reset process does not clean CNI configuration. To do so, you must remove /etc/cni/net.d

The reset process does not reset or clean up iptables rules or IPVS tables.
If you wis

In [9]:
# Expressões regulares para encontrar os valores após "--token" e "--discovery-token-ca-cert-hash"
padrao_token = r"--token\s+([\w\.]+)"
padrao_cert_hash = r"--discovery-token-ca-cert-hash\s+(\S+)"

# Encontrar os valores usando expressões regulares
token_encontrado = re.search(padrao_token, key_string)
cert_hash_encontrado = re.search(padrao_cert_hash, key_string)

# Verificar se os valores foram encontrados e atribuí-los às variáveis correspondentes
if token_encontrado:
    token = token_encontrado.group(1)
else:
    token = None

if cert_hash_encontrado:
    cert_hash = cert_hash_encontrado.group(1)
else:
    cert_hash = None

# Imprimir os valores encontrados
print("Token:", token)
print("Cert Hash:", cert_hash)

Token: ac1bzj.ubmns7ol3z6y5k48
Cert Hash: sha256:1945f803c1ab599db17949baaf8ca278a6b3771781d0a0fa78b5188bccc619b8


### Configuring Kubernetes Workers

In [10]:
try:
    
    file_attributes = node2.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_worker_node-v2.sh", remote_file_path="config_worker_node-v2.sh")
    file_attributes = node3.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_worker_node-v2.sh", remote_file_path="config_worker_node-v2.sh")
    file_attributes = node4.upload_file(local_file_path="/home/fabric/work/kubernetes_files/config_worker_node-v2.sh", remote_file_path="config_worker_node-v2.sh")
    
    stdout, stderr = node2.execute(f"chmod +x config_worker_node-v2.sh && ./config_worker_node-v2.sh")
    stdout, stderr = node3.execute(f"chmod +x config_worker_node-v2.sh && ./config_worker_node-v2.sh")
    stdout, stderr = node4.execute(f"chmod +x config_worker_node-v2.sh && ./config_worker_node-v2.sh")
    
except Exception as e:
    print(f"Exception: {e}")



Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3187 kB]
Get:12 http://nova.clouds.arch

## Starting Workers Nodes

In [11]:
try:
    file_attributes = node2.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_worker_node-v2.sh", remote_file_path="start_worker_node-v2.sh")
    file_attributes = node3.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_worker_node-v2.sh", remote_file_path="start_worker_node-v2.sh")
    file_attributes = node4.upload_file(local_file_path="/home/fabric/work/kubernetes_files/start_worker_node-v2.sh", remote_file_path="start_worker_node-v2.sh")
    
    
    #file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/dns-ipv6.yaml", remote_file_path="dns-ipv6.yaml")
    #stdout, stderr = node1.execute(f"kubectl apply -f dns-ipv6.yaml")
    #file_attributes = node2.upload_file(local_file_path="/home/fabric/work/kubernetes_files/dns-ipv6.yaml", remote_file_path="dns-ipv6.yaml")
    #stdout, stderr = node2.execute(f"kubectl apply -f dns-ipv6.yaml")
    #file_attributes = node3.upload_file(local_file_path="/home/fabric/work/kubernetes_files/dns-ipv6.yaml", remote_file_path="dns-ipv6.yaml")
    #stdout, stderr = node3.execute(f"kubectl apply -f dns-ipv6.yaml")
    #file_attributes = node4.upload_file(local_file_path="/home/fabric/work/kubernetes_files/dns-ipv6.yaml", remote_file_path="dns-ipv6.yaml")
    #stdout, stderr = node4.execute(f"kubectl apply -f dns-ipv6.yaml")
    
    stdout, stderr = node2.execute(f"chmod +x start_worker_node-v2.sh && ./start_worker_node-v2.sh {node1_addr} {token} {cert_hash}")
    stdout, stderr = node3.execute(f"chmod +x start_worker_node-v2.sh && ./start_worker_node-v2.sh {node1_addr} {token} {cert_hash}")
    stdout, stderr = node4.execute(f"chmod +x start_worker_node-v2.sh && ./start_worker_node-v2.sh {node1_addr} {token} {cert_hash}")
except Exception as e:
    print(f"Exception: {e}")

10.136.8.2
W0321 22:37:14.733253    6962 preflight.go:56] [reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
W0321 22:37:14.733296    6962 removeetcdmember.go:106] [reset] No kubeadm config, using etcd pod spec to get data directory
[reset] Deleted contents of the etcd data directory: /var/lib/etcd
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
[reset] Deleting contents of directories: [/etc/kubernetes/manifests /var/lib/kubelet /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]

The reset process does not clean CNI configuration. To do so, you must remove /etc/cni/net.d

The reset process does not reset or clean up iptables rules or IPVS tables.

## Getting Cluster Information

In [12]:
try:
    stdout, stderr = node1.execute("kubectl get pods --all-namespaces")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAMESPACE          NAME                                      READY   STATUS              RESTARTS   AGE
calico-apiserver   calico-apiserver-67f868f997-g8d99         1/1     Running             0          118s
calico-apiserver   calico-apiserver-67f868f997-pkmgp         1/1     Running             0          118s
calico-system      calico-kube-controllers-94fd6f7d9-jwzd7   1/1     Running             0          2m32s
calico-system      calico-node-brfxc                         1/1     Running             0          2m32s
calico-system      calico-node-d8f87                         0/1     Init:0/2            0          2s
calico-system      calico-node-thrcs                         0/1     PodInitializing     0          11s
calico-system      calico-node-v69vq                         0/1     Init:0/2            0          6s
calico-system      calico-typha-fc565664f-mq7j6              0/1     ContainerCreating   0          3s
calico-system      calico-typha-fc565664f-tz9vb              

# Cassandra Cluster

In [ ]:
try:
    stdout, stderr = node1.execute("git clone https://github.com/romoreira/SFI2-Distributed-Orchestration.git")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
    stdout, stderr = node1.execute("kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/svc-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/sc.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/pv-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/pvc-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/sts-cassandra.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/load_gen/stress.yaml")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

## Installing Helm 2.0X

In [13]:
try:
    stdout, stderr = node1.execute("sudo curl -LO https://git.io/get_helm.sh")
    stdout, stderr = node1.execute("sudo chmod 700 get_helm.sh")
    stdout, stderr = node1.execute("sudo ./get_helm.sh")
    stdout, stderr = node1.execute("helm version --client")
    stdout, stderr = node1.execute("helm init")
    stdout, stderr = node1.execute("git clone https://github.com/romoreira/OrcaReprod.git")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
    
    

   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100  6875  100  6875    0     0   1924      0  0:00:03  0:00:03 --:--:--  1924
Preparing to install helm and tiller into /usr/local/bin
helm installed into /usr/local/bin/helm
tiller installed into /usr/local/bin/tiller
Run 'helm init' to configure helm.
Client: &version.Version{SemVer:"v2.17.0", GitCommit:"a690bad98af45b015bd3da1a41f6218b1a451dbe", GitTreeState:"clean"}
Creating /home/ubuntu/.helm 
Creating /home/ubuntu/.helm/repository 
Creating /home/ubuntu/.helm/repository/cache 
Creating /home/ubuntu/.helm/repository/local 
Creating /home/ubuntu/.helm/plugins 
Creating /home/ubuntu/.helm/starters 
Creating /home/ubuntu/.helm/cache/archive 
Creating /home/ubuntu/.helm/repository/repositories.yaml 
Adding stable repo with URL: https://charts.helm.sh

## See Running Pods

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl get pods -o wide")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## See Cassandra Cluster

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl logs cassandra-0")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## See Running Pods - Cassandra

In [ ]:
try:
    stdout, stderr = node1.execute("kubectl exec -it cassandra-0 -- nodetool status")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

## Installing Helm on each node of Kubernetes

In [ ]:

try:
       
    stdout, stderr = node1.execute(f"sudo snap install helm --classic")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
    #stdout, stderr = node2.execute(f"sudo snap install helm --classic")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #stdout, stderr = node3.execute(f"sudo snap install helm --classic")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #stdout, stderr = node4.execute(f"sudo snap install helm --classic")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Installing K-Sniff to Collect Packets of Cassandra Pods

In [ ]:
try:
    file_attributes = node1.upload_file(local_file_path="/home/fabric/work/kubernetes_files/krew_installation.sh", remote_file_path="krew_installation.sh")
    
    stdout, stderr = node1.execute(f"chmod +x krew_installation.sh && ./krew_installation.sh")

except Exception as e:
    print(f"Exception: {e}")

## Installing Helm

In [ ]:
try:
        
    stdout, stderr = node1.execute(f"sudo snap install helm --classic ; helm repo add netdata https://netdata.github.io/helmchart/ ; helm install netdata netdata/netdata")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
    #stdout, stderr = node2.execute(f"sudo snap install helm --classic ; helm repo add netdata https://netdata.github.io/helmchart/ ; helm install netdata netdata/netdata")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #stdout, stderr = node3.execute(f"sudo snap install helm --classic ; helm repo add netdata https://netdata.github.io/helmchart/ ; helm install netdata netdata/netdata")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #stdout, stderr = node4.execute(f"sudo snap install helm --classic ; helm repo add netdata https://netdata.github.io/helmchart/ ; helm install netdata netdata/netdata")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")

except Exception as e:
    print(f"Exception: {e}")

## Deploying NetData Persistent Volumes

In [ ]:
try:
    #stdout, stderr = node1.execute(f"git clone https://github.com/romoreira/SFI2-Distributed-Orchestration.git")
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    stdout, stderr = node1.execute(f"kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/netdata/storageclass.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/netdata/pv-netdata-alarms.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/netdata/pv-netdata-database.yaml ; kubectl apply -f /home/ubuntu/SFI2-Distributed-Orchestration/3rd/netdata/pv-netdata-k8s-state.yaml")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Updating Helm to support NetData

In [ ]:
try:
    stdout, stderr = node1.execute(f"helm upgrade --set parent.database.storageclass=default netdata netdata/netdata")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
    stdout, stderr = node1.execute(f"helm upgrade --set parent.alarms.storageclass=default netdata netdata/netdata")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
    stdout, stderr = node1.execute(f"helm upgrade --set k8sState.persistence.storageclass=default netdata netdata/netdata")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

## Checking NetData Agents

In [ ]:
try:
    
    stdout, stderr = node1.execute(f"kubectl get pods -o wide")
    print(stdout, stderr)
except Exception as e:
    print(f"Exception: {e}")

## Enabling SSH Tunnel

In [ ]:
# Port on your local machine that you want to map the Netdata to.
local_port='19999'
# Local interface to map the File Browser to (can be `localhost`)
local_host='127.0.0.1'

# Port on the node used by the File Browser Service
target_port='19999'

# Username/node on FABRIC
target_host=f'{node1.get_username()}@{node1.get_management_ip()}'

print(f'ssh  -L {local_host}:{local_port}:127.0.0.1:{target_port} -i slice_key -F ssh_config {target_host}')


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")